In [34]:
import numpy as np
import pandas as pd
import os

#### MetaData

In [35]:
# read and merge csv files (Weather)
read_weather = True
# process and save processed csv files (Weather)
process_weather = True
station_name = 'Ardenville'


# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# GENERATION
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# read and merge csv files (Generation)
read_generation = True
# read and merge csv files (Generation)
save_process_generation = True
asset_name = "AKE1 McBride Lake Windfarm"


In [36]:
def clean_df(file_path: os.PathLike):
    df = pd.read_csv(file_path, encoding='unicode_escape')  # Read CSV file with proper encoding
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # Remove any "Unnamed" columns that might be auto-generated
    return df  # Return the cleaned DataFrame

## Weather Data

#### Read csv files and merge them

In [37]:
# Read CSVs and drop any unnamed index column
if read_weather:
    # Create a new folder to save the merged CSV file
    os.makedirs('./Data/Merged', exist_ok=True)
    # Load cleaned CSVs (each file contains data for a specific time period)
    file_path = f"./Data/ACIS/{station_name}"
    all_weather_data = pd.concat([clean_df(f'{file_path}/{file}') for file in os.listdir(f'{file_path}')], ignore_index=True)
    # Save the merged dataframe as a new CSV file without index values
    all_weather_data.to_csv(f"./Data/Merged/{station_name}_weather_data.csv", index=False)  # `index=False` prevents writing index column to CSV
    print('Merged CSV file saved successfully!')
    print(f"Merged data has {all_weather_data.shape[0]} rows.")

Merged CSV file saved successfully!
Merged data has 43843 rows.


In [38]:
def process_weather_station_data(df: pd.DataFrame, save: bool = False, columns = ['Date', 'Air Temp. Inst. (Â°C)', 'Humidity Inst. (%)', 'Incoming Solar Rad. (W/m2)','Precip. (mm)', 'Wind Speed 10 m Syno. (km/h)', 'Wind Dir. 10 m Syno. (Â°)', 'Wind Speed 10 m Avg. (km/h)', 'Wind Dir. 10 m Avg. (Â°)',]) -> pd.DataFrame:
    """
    Get data for a specific asset.
    """
    try:
        print(df.columns)
        df['Date'] = pd.to_datetime(df['Date (Local Standard Time)'], errors='coerce')        
        columns_to_drop = [column for column in df.columns if column not in columns]
        df.drop(columns=columns_to_drop, axis=1, inplace=True)
        print(df.isna().sum())
        df.fillna(method='ffill', inplace=True)
        if save:
            df.to_csv(f'./Data/Merged/{station_name}_weather.csv', index=False)
            print('Processed CSV file saved successfully!')
        print("Miissing values filled")
        print(f"Data for {station_name} has {df.shape[0]} rows.")
        return df
    except Exception as e:
        print(f"No data found for {station_name}. Error: {e}")

#### Process Data

In [39]:
# Load the merged CSV file
weather_data = clean_df(f"./Data/Merged/{station_name}_weather_data.csv")
if process_weather:
    station_data = process_weather_station_data(weather_data, save=True)
    # Convert Data
    print(station_data)
    # Create a new folder to save the merged CSV file
    os.makedirs('./Data/Merged', exist_ok=True)
    # Fill missing values with the average of previous and next values
    # print(station_data.isna().sum())
    # print(f"Processed data has {station_data.shape[0]} rows.")

C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\3648812536.py:2: DtypeWarning: Columns (7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='unicode_escape')  # Read CSV file with proper encoding
C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\1026872937.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


Index(['Station Name', 'Date (Local Standard Time)', 'Air Temp. Inst. (Â°C)',
       'Air Temp. Inst. Source Flag', 'Air Temp. Inst. Comment',
       'Humidity Inst. (%)', 'Humidity Inst. Source Flag',
       'Humidity Inst. Comment', 'Relative Humidity Avg. (%)',
       'Relative Humidity Avg. Source Flag', 'Relative Humidity Avg. Comment',
       'Incoming Solar Rad. (W/m2)', 'Incoming Solar Rad. Source Flag',
       'Incoming Solar Rad. Comment', 'Precip. (mm)', 'Precip. Source Flag',
       'Precip. Comment', 'Wind Speed 10 m Syno. (km/h)',
       'Wind Speed 10 m Syno. Source Flag', 'Wind Speed 10 m Syno. Comment',
       'Wind Dir. 10 m Syno. (Â°)', 'Wind Dir. 10 m Syno. Source Flag',
       'Wind Dir. 10 m Syno. Comment', 'Wind Speed 10 m Avg. (km/h)',
       'Wind Speed 10 m Avg. Source Flag', 'Wind Speed 10 m Avg. Comment',
       'Wind Dir. 10 m Avg. (Â°)', 'Wind Dir. 10 m Avg. Source Flag',
       'Wind Dir. 10 m Avg. Comment'],
      dtype='object')
Air Temp. Inst. (Â°C)   

## Generation Data

#### Read csv files and merge them

In [40]:
if read_generation:
    # Load data using a for loop and a list comprehension
    generation_data = pd.concat([clean_df(f'./Data/CSD/{file}') for file in os.listdir('./Data/CSD')], ignore_index=True)
    # Create a new folder to save the merged CSV file
    os.makedirs('./Data/Merged', exist_ok=True)
    # Save the merged dataframe as a new CSV file without index values
    generation_data.to_csv('./Data/Merged/Generation.csv', index=False)  # `index=False` prevents writing index column to CSV
    print('Merged CSV file saved successfully!')
    print(f"Merged data has {generation_data.shape[0]} rows.")

Merged CSV file saved successfully!
Merged data has 4866764 rows.


#### Data Processing

In [41]:
# Find assets with most data in the generation data
def get_assets_with_most_data(df: pd.DataFrame, fuel_type: str) -> pd.Series:
    """
    Get the assets with the most data points in the generation data.
    """
    try:
        df = df[df['Fuel Type'] == fuel_type.upper()]
        asset_list = df['Asset Name'].value_counts()
        most_valuable_asset = asset_list.where(asset_list == asset_list[0]).dropna()
        print(f"Total number of assets: {len(asset_list)}")
        print(f"{len(most_valuable_asset)} assets have the most data. They have {most_valuable_asset[0]} data points.")
        return most_valuable_asset
    except Exception as e:
        print(f"No data found for {fuel_type} assets.")

In [42]:
# Get data for a specific asset.
def get_asset_data(df: pd.DataFrame, asset_name: str, save: bool = False, columns = ['Volume', 'Maximum Capability', 'System Capability']) -> pd.DataFrame:
    """
    Get data for a specific asset.
    """
    try:
        asset_data = df[df['Asset Name'] == asset_name]
        columns_to_drop = [column for column in asset_data.columns if column not in columns]
        asset_data['Date'] = pd.to_datetime(asset_data['Date (MST)'], errors='coerce')
        asset_data.drop(columns=columns_to_drop, axis=1, inplace=True)
        print(asset_data.isna().sum())
        asset_data.fillna(method='ffill', inplace=True)
        if save:
            asset_data.to_csv(f'./Data/Merged/{asset_name}_generation.csv', index=False)
            print('Processed CSV file saved successfully!')
        print("Miissing values filled")
        print(f"Data for {asset_name} has {asset_data.shape[0]} rows.")
        return asset_data
    except Exception as e:
        print(f"No data found for {asset_name}.")

In [43]:
# Read merged data
generation_data = clean_df('./Data/Merged/Generation.csv')

In [44]:
get_assets_with_most_data(generation_data,'wind')
# x = generation_data.where(generation_data["Planning Area"] == 47).dropna()
# x.where(x["Fuel Type"] == "WIND").dropna()
# generation_data.where(generation_data['Fuel Type'] == 'SOLAR').dropna()


Total number of assets: 51
26 assets have the most data. They have 26304.0 data points.


C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\2297823499.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  most_valuable_asset = asset_list.where(asset_list == asset_list[0]).dropna()
C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\2297823499.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"{len(most_valuable_asset)} assets have the most data. They have {most_valuable_asset[0]} data points.")


Asset Name
AKE1 McBride Lake Windfarm                 26304.0
ARD1 Ardenville                            26304.0
BSR1 Blackspring Ridge                     26304.0
BTR1 Blue Trail Wind                       26304.0
BUL1 Bull Creek                            26304.0
BUL2 Bull Creek                            26304.0
CR1 ARM2262 Castle River                   26304.0
Cowley Ridge                               26304.0
CRR1 Enel Alberta Castle Rock Wind Farm    26304.0
CRR2 Castle Rock Ridge 2                   26304.0
GWW1 GW Wind #1                            26304.0
HAL1 Halkirk Wind Power Facility           26304.0
IEW1 Summerview Phase 1                    26304.0
IEW2 Summerview Phase 2                    26304.0
KHW1 Kettles Hill Wind                     26304.0
NEP1 Ghost Pine                            26304.0
OWF1 Oldman 2 Wind Farm 1                  26304.0
RIV1 Riverview                             26304.0
RTL1 Rattlesnake Ridge Wind                26304.0
SCR2 Magrath        

In [45]:
asset_gen_data = get_asset_data(generation_data, asset_name, save_process_generation)

Volume                0
Maximum Capability    0
System Capability     0
Date                  0
dtype: int64
Processed CSV file saved successfully!
Miissing values filled
Data for AKE1 McBride Lake Windfarm has 26304 rows.


C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\1683497974.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data['Date'] = pd.to_datetime(asset_data['Date (MST)'], errors='coerce')
C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\1683497974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asset_data.drop(columns=columns_to_drop, axis=1, inplace=True)
C:\Users\nabil\AppData\Local\Temp\ipykernel_32852\1683497974.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() inste

## Merge Data

In [46]:
merged_df = pd.merge(asset_gen_data, weather_data, on='Date', how='inner')
merged_rows = merged_df.shape[0]
print(f"The merged file has {merged_rows} rows.")
print(merged_df.isna().sum())
merged_df.to_csv(f"./Data/Merged/{asset_name}_Processed_and_Data.csv", index=False)  # `index=False` prevents writing index column to CSV

The merged file has 26301 rows.
Volume                          0
Maximum Capability              0
System Capability               0
Date                            0
Air Temp. Inst. (Â°C)           0
Humidity Inst. (%)              0
Incoming Solar Rad. (W/m2)      0
Precip. (mm)                    0
Wind Speed 10 m Syno. (km/h)    0
Wind Dir. 10 m Syno. (Â°)       0
Wind Speed 10 m Avg. (km/h)     0
Wind Dir. 10 m Avg. (Â°)        0
dtype: int64
